In [1]:
# On this notebook I'll build the SVC model 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Read the data
cardio_data=pd.read_csv("in/cardio_train.csv")
cardio_data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
cardio_data.groupby("cardio").size()
# The classes are already balanced

cardio
0    35021
1    34979
dtype: int64

In [4]:
# Split data into features and labels
X=cardio_data.iloc[:,1:12].values
y=cardio_data["cardio"].values

In [5]:
# Standarization and spliting process
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# Also I'll split the dataset in val test, to have 3 sets from the initial one.
# X train = 56000 / X test = 14000 / X val = 14000
# With partial train I'll train the model, with test I'll improve the model and with test I'll evaluate the model
X_val = X_train[:14000]
partial_X_train = X_train[14000:]
X_test = X_test

y_val = y_train[:14000]
partial_y_train = y_train[14000:]
y_test = y_test

In [7]:
# Standarization of variables
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
X_train_std=std.fit_transform(partial_X_train)

In [8]:
# Remeber differences between fit and transform
X_test_std = std.transform(X_test)
X_val_std = std.transform(X_val)

In [10]:
# Applying the SVC
from sklearn.ensemble  import RandomForestClassifier
classifier_RFC = RandomForestClassifier().fit(X_train_std, partial_y_train)

In [11]:
# Predicting to evaluate
y_test_predicted = classifier_RFC.predict(X_test_std)
y_training_predicted = classifier_RFC.predict(X_train_std)

In [12]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
print("Training Accuracy:",metrics.accuracy_score(partial_y_train, y_training_predicted))
print("Testing Accuracy:",metrics.accuracy_score(y_test, y_test_predicted))

Training Accuracy: 0.9998571428571429
Testing Accuracy: 0.7129285714285715


In [13]:
# Confunsion matrix
cnf_matrix_training = metrics.confusion_matrix(partial_y_train, y_training_predicted)
cnf_matrix_testing = metrics.confusion_matrix(y_test,y_test_predicted)
print("The confunsion matrix results for the training part are:")
print(cnf_matrix_training)
print("The confunsion matrix results for the testing part are:")
print(cnf_matrix_testing)

The confunsion matrix results for the training part are:
[[21008     2]
 [    4 20986]]
The confunsion matrix results for the testing part are:
[[5087 1934]
 [2085 4894]]


In [14]:
# cross validation to analize the accuraccy
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold_val = KFold(5)
cross_results = cross_val_score(classifier_RFC, X_train_std, partial_y_train, cv=kfold_val)

print(cross_results)
print(cross_results.mean())

[0.71488095 0.71261905 0.71369048 0.71440476 0.71535714]
0.7141904761904762


In [ ]:
# In this case I found the first difference between SVC and RFC. It is RFC has a better accuracy in the training data
# But, in comparison with the test data is not the best. So here, we have overfitting. But, at the same time. 
# We have a better performance.